In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

In [3]:
data_path = r"D:\agentic_ai_and_genai\GEN_AI_Assignments\data"
loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [4]:
len(documents)

205

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
semantic_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [6]:
from langchain_experimental.text_splitter import SemanticChunker
text_splitter = SemanticChunker(semantic_embeddings, breakpoint_threshold_type="percentile")
chunks = text_splitter.split_documents(documents)

#### setup milvus locally windows
#### run docker desktop as Administrator and use the docker terminal inside it.
```bash 
Invoke-WebRequest https://raw.githubusercontent.com/milvus-io/milvus/refs/heads/master/scripts/standalone_embed.bat  -OutFile standalone.bat

./standalone.bat start

# after this restart it
./standalone.bat restart

./standalone.bat stop
./standalone.bat delete
```

In [ ]:
from langchain_milvus import Milvus
from uuid import uuid4

# Create a unique collection name for each index type later
def get_collection_name(index_type):
    return f"rag_collection_{index_type}_{uuid4().hex}"

print("Milvus store ready")

Milvus store ready


In [9]:
from pymilvus import utility, connections

In [10]:
# Connect to Milvus
connections.connect(alias="default", host="localhost", port="19530")
connections


In [11]:
# Define index types and create Milvus stores
index_configs = {
    "FLAT": {"index_type": "FLAT", "metric_type": "L2"},
    "HNSW": {"index_type": "HNSW", "metric_type": "L2", "params": {"M": 16, "efConstruction": 100}},
    "IVF": {"index_type": "IVF_FLAT", "metric_type": "L2", "params": {"nlist": 100}}
}

In [12]:
# Dictionary to store vector store and collection name for each index type
collections = {}

for idx_type, params in index_configs.items():
    collection_name = get_collection_name(idx_type)
    
    # Drop existing collection if it exists
    if utility.has_collection(collection_name):
        print(f"Dropping existing collection: {collection_name}")
        utility.drop_collection(collection_name)

    print(f"Creating collection: {collection_name} with index: {idx_type}")
    
    vector_store = Milvus(
        embedding_function=semantic_embeddings,
        connection_args={"host": "localhost", "port": "19530"},
        index_params=params,
        collection_name=collection_name,
        enable_dynamic_field=True
    )
    
    vector_store.add_documents(chunks)

    # Store both the vector store and collection name
    collections[idx_type] = {
        "store": vector_store,
        "collection_name": collection_name
    }

print("All indexes created and data inserted into Milvus")
print("HNSW Collection Name:", collections['HNSW']['collection_name'])


2025-06-02 22:22:07,748 [DEBUG][_create_connection]: Created new connection using: 1562712f26dc442fa6e0d99bf3e55ca1 (async_milvus_client.py:599)


Creating collection: rag_collection_FLAT_4e2e261a89b24084a78b6f6dcb54cd95 with index: FLAT


d:\agentic_ai_and_genai\.venv\lib\site-packages\langchain_milvus\vectorstores\milvus.py:1141: UserWarning: No ids provided and auto_id is False. Setting auto_id to True automatically.
  warnings.warn(
2025-06-02 22:22:49,769 [DEBUG][_create_connection]: Created new connection using: d9acd5284e434c658cdbf33277686c11 (async_milvus_client.py:599)


Creating collection: rag_collection_HNSW_f239b3c846174110817b4c885ea5f6ef with index: HNSW


2025-06-02 22:23:31,605 [DEBUG][_create_connection]: Created new connection using: 2799d5c4aa304249a6ad3ca7fc901db7 (async_milvus_client.py:599)


Creating collection: rag_collection_IVF_35676091663845fa909758d434c506a3 with index: IVF
All indexes created and data inserted into Milvus
HNSW Collection Name: rag_collection_HNSW_f239b3c846174110817b4c885ea5f6ef


In [17]:
collections['HNSW']

{'store': <langchain_milvus.vectorstores.milvus.Milvus at 0x1d871465c60>,
 'collection_name': 'rag_collection_HNSW_f239b3c846174110817b4c885ea5f6ef'}

In [18]:
collections['HNSW']['store']

In [22]:
# Set up retrievers for each index
retrievers = {}

for idx_type, data in collections.items():
    store = data["store"]
    retriever = store.as_retriever()
    retrievers[idx_type] = retriever


In [24]:
# Retrieval speed
import time

query = "What is graph neural network?"


results = {}
for idx_type, retriever in retrievers.items():
    start_time = time.time()
    docs = retriever.invoke(query)
    elapsed = time.time() - start_time
    results[idx_type] = {"time": elapsed, "docs": docs}
    print(f"{idx_type}: Retrieved in {elapsed:.4f}s")

FLAT: Retrieved in 3.7172s
HNSW: Retrieved in 0.4745s
IVF: Retrieved in 1.2689s


In [25]:
from langchain_core.documents import Document
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
query = "What is graph neural network"
query_embedding = semantic_embeddings.embed_query(query)

In [ ]:
def get_relevance_score(query_emb, doc: Document):
    # Embed the document content
    doc_embedding = semantic_embeddings.embed_documents([doc.page_content])[0]
    
    # Compute cosine similarity
    sim = cosine_similarity([query_emb], [doc_embedding])[0][0]
    return sim

In [28]:
# Evaluate accuracy
scores = {}
for idx_type, result in results.items():
    total = len(result["docs"])
    relevant = sum(
        1 for doc in result["docs"]
        if get_relevance_score(query_embedding, doc) > 0.7  
    )
    scores[idx_type] = relevant / total if total > 0 else 0

print("Accuracy Scores:", scores)


Accuracy Scores: {'FLAT': 1.0, 'HNSW': 1.0, 'IVF': 1.0}


In [31]:
#  Use MMR re-ranking
mmr_results = {}

for idx_type, data in collections.items():
    store = data["store"]
    mmr_docs = store.max_marginal_relevance_search(query, k=9, lambda_mult=0.5)
    mmr_results[idx_type] = mmr_docs
    print(f"[{idx_type}] MMR Results:\n{[d.page_content[:100] for d in mmr_docs]}...\n")

[FLAT] MMR Results:
['XX, AUGUST 2019 1\nA Comprehensive Survey on Graph Neural\nNetworks\nZonghan Wu, Shirui Pan, Member, IE', 'Neural Networks 173 (2024) 106207\n7\nW. Ju et al. where 𝜖 is a learnable parameter. More recently, ef', 'Unlike\nother neural network architectures, GNNs can handle\nnon-euclidean data with complex relations', '[\n𝐴(𝑙)\n(𝑟)(𝐺,𝐺′)\n]\n𝑢𝑢′\n=\n⎛\n⎜\n⎜\n⎜\n⎜\n⎜⎝\n[ (𝑙)∑\n(𝑟−1)\n(𝐺,𝐺)\n]\n𝑢𝑢′\n,\n[ (𝑙)∑\n(𝑟−1)\n(𝐺,𝐺′)\n]\n𝑢𝑢′\n[ (𝑙)∑\n(𝑟', '17.2.2. Combining spectral graph theory\nDown to the theory foundations, the idea of graph neural net', 'IV. R ECURRENT GRAPH NEURAL NETWORKS\nRecurrent graph neural networks (RecGNNs) are mostly pi-\noneer ', 'For\neach node, the model takes a weighted sum of its neighbors’ features\nas well as its own features', 'XX, NO. XX, AUGUST 2019 3\nnetwork embedding, another topic which attracts increasing\nattention from ', 'They\nassume the pre-deﬁned graph structure reﬂects the genuine\ndependency relationships among nodes.']...


In [32]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [33]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17", temperature=0.3)

In [34]:
collections

{'FLAT': {'store': <langchain_milvus.vectorstores.milvus.Milvus at 0x1d87181b310>,
  'collection_name': 'rag_collection_FLAT_4e2e261a89b24084a78b6f6dcb54cd95'},
 'HNSW': {'store': <langchain_milvus.vectorstores.milvus.Milvus at 0x1d871465c60>,
  'collection_name': 'rag_collection_HNSW_f239b3c846174110817b4c885ea5f6ef'},
 'IVF': {'store': <langchain_milvus.vectorstores.milvus.Milvus at 0x1d8760c47f0>,
  'collection_name': 'rag_collection_IVF_35676091663845fa909758d434c506a3'}}

In [35]:
vector_store = collections['HNSW']['store']

In [36]:
vector_store

In [38]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7, "k": 5}
)

In [39]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [40]:
prompt=PromptTemplate(
    template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:""",
    input_variables=['context', 'question']
)

In [41]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [42]:
query = "What is graph neural network?"
response = rag_chain.invoke(query)

response

'Graph Neural Networks (GNNs) are a type of neural network designed to handle data represented as graphs, which have complex relationships and interdependencies between objects in non-Euclidean domains. Unlike other neural networks, GNNs can process this non-Euclidean data. Most GNNs follow a message-passing framework and can be considered a generalization of convolutional neural networks for graphs.'

In [43]:
from docx import Document

doc = Document()
doc.add_heading("RAG Response", level=1)
doc.add_paragraph(response)

doc.save("rag_output.docx")
print("Answer saved to rag_output.docx")

Answer saved to rag_output.docx
